In [ ]:
from exif import Image
from pathlib import Path

img = Path("/home/pierre/Pictures/Phone-Camera/PXL_20230924_015614422.jpg")
d = Image(open(img, "rb"))

print(d.get("flash"))

def debug(k):
    print(k)
    return k
{
    k: str(d.get(k))
    for k in dir(d)
    if not k.startswith("_") and not callable(d.get(k)) and d.get(k) is not None
}

In [ ]:
from operator import add, sub, mul

s = "- * 3 + 1 2 4"

def bob(l):
    s = l.pop(0)
    if s.isnumeric():
        return int(s)
    return {"+": add, "-": sub, "*": mul}[s](bob(l), bob(l))

bob(s.split())


In [ ]:
from operator import add, sub, mul

s = "- * 3 + 1 2 4"

def bob(l):
    return int(s) if (s:=l.pop(0)).isnumeric() else {"+": add, "-": sub, "*": mul}[s](bob(l), bob(l))

bob(s.split())


In [ ]:
import PIL.ExifTags
import PIL.Image

d = dict(zip(PIL.ExifTags.TAGS.values(), PIL.ExifTags.TAGS.keys()))
d["UserComment"]

img = PIL.Image.open("/home/pierre/repositories/exif-tagger/res/PXL_20231217_005734707.jpg")

In [ ]:
exif = img.getexif()
{
    PIL.ExifTags.TAGS[k]: v
    for k,v in exif.items()
    if k in PIL.ExifTags.TAGS
}

In [ ]:
ifd = exif.get_ifd(PIL.ExifTags.IFD.GPSInfo)
{
    PIL.ExifTags.GPSTAGS[k]: v
    for k,v in ifd.items()
    if k in PIL.ExifTags.GPSTAGS
}

In [ ]:
import pandas as pd
import json

df = pd.DataFrame(
    {"a": list(range(5)), "b": list(range(5,10))}
)

for i,row in df.iterrows():
    print(json.dumps(dict(row)))
    # print(dict(row))


In [ ]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    surname: str

Person(*["pierre","louvart"])

In [ ]:
from exif_tagger.database import FaceDatabase,Person
from exif_tagger.img_clipper import infer

In [ ]:
pierre = Person(name="Pierre", surname="Louvart")
henri = Person(name="Henri", surname="Louvart")

db = FaceDatabase(database_path="../db/face-database.sqlite")

face = db.getFaces()[1]
infer(db, face)


In [1]:
from exif_tagger.embeddings import generate_embeddings
embs,names = generate_embeddings("/home/pierre/Downloads/VGG-Face2/data/vggface2_test/test")

/home/pierre/repositories/exif-tagger/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on device: cpu


  0%|          | 3/1694 [00:28<4:23:35,  9.35s/it]


KeyboardInterrupt: 

In [ ]:
import numpy as np
np.save("../res/embeddings_vg2.npy", embs.numpy())

In [8]:
import json
json.dump(names, open("../res/names.json", "w"))

In [2]:
import numpy as np
import json

names = json.load(open("../res/names.json"))

embs = np.load("../res/embeddings_vg2.npy")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
embs_9 = embs[np.array(names) == "n000009"] # Asian
embs_29 = embs[np.array(names) == "n000029"] # Old White
embs_40 = embs[np.array(names) == "n000040"] # Old White
embs_480 = embs[np.array(names) == "n000480"] # White Woman
embs_2351 = embs[np.array(names) == "n002351"] # Black
embs_928 = embs[np.array(names) == "n000928"] # Black

In [32]:
print(
    np.sum((np.mean(embs_9, axis=0)- np.mean(embs_29, axis=0))**2)**.5,
    np.sum((np.mean(embs_29, axis=0)- np.mean(embs_40, axis=0))**2)**.5,
    np.sum((np.mean(embs_9, axis=0)- np.mean(embs_40, axis=0))**2)**.5,
    np.sum((np.mean(embs_9, axis=0)- np.mean(embs_480, axis=0))**2)**.5,
    np.sum((np.mean(embs_40, axis=0)- np.mean(embs_480, axis=0))**2)**.5,
    np.sum((np.mean(embs_40, axis=0)- np.mean(embs_2351, axis=0))**2)**.5,
    np.sum((np.mean(embs_928, axis=0)- np.mean(embs_2351, axis=0))**2)**.5,
)

0.9663953159117706 0.8319970782072776 0.9341694481360491 1.0346331732292187 1.03297563150399 1.1046451155801678 0.8550903667969524


In [35]:
np.sum((embs[np.array(names) == "n000009"][::2].mean(axis=0)-
embs[np.array(names) == "n000009"][1::2].mean(axis=0))**2)**.5


0.14762818407828815